# Import Library

In [145]:
import os
import re
import json
import xml.etree.ElementTree as ET
from pathlib import Path

import pandas as pd
import numpy as np

# Functions

In [9]:
def get_namespace(element):
    m = re.match(r'\{.*\}', element.tag)
    if m:
        tag = m.group(0)
    else:
        tag = ''
    return  tag

In [75]:
def parse_data(root):
    list_records = list()
    xml_nms = get_namespace(root)

    list_elems = root.findall(f'{xml_nms}logitem')
    n_all_elems = len(list_elems)
    nth_percent_display = n_all_elems // 10

    xml_tag_mapper = {
        f'{xml_nms}id': 'log_id',
        f'{xml_nms}timestamp': 'log_timestamp',
        f'{xml_nms}contributor/{xml_nms}username': 'user_name',
        f'{xml_nms}contributor/{xml_nms}id': 'user_id',
        f'{xml_nms}comment': 'user_comment',
        f'{xml_nms}type': 'action_type',
        f'{xml_nms}action': 'action',
        f'{xml_nms}logtitle': 'log_title'
    }

    for i, elem in enumerate(list_elems):
        record = dict()

        for xml_tag, col_name in xml_tag_mapper.items():
            try:
                value = elem.find(xml_tag).text
            except AttributeError:
                value = None
            
            record[col_name] = value

        list_records.append(record)

        if i % nth_percent_display == 0:
            progress = int(i / nth_percent_display)
            if progress <= 9 and progress > 0:
                print("progress: {}% {}".format(10 * progress, "." * progress))
            else:
                if i == 10:
                    print("done")

    df = pd.read_json(json.dumps(list_records), orient='records')

    return df

# Import Data

In [2]:
path = Path(os.getcwd())

project_dir = path.parent
raw_data_dir = project_dir / 'data' / 'raw'

In [3]:
tree_page_logging = ET.parse(raw_data_dir / 'thwiki-20240920-pages-logging.xml')
root_page_logging = tree_page_logging.getroot()

In [76]:
df_user_log = parse_data(root=root_page_logging)

progress: 10% .
progress: 20% ..
progress: 30% ...
progress: 40% ....
progress: 50% .....
progress: 60% ......
progress: 70% .......
progress: 80% ........
progress: 90% .........


C:\Users\11413929\AppData\Local\Temp\ipykernel_18484\3256056291.py:40: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json.dumps(list_records), orient='records')


In [80]:
df_user_log.head(3)

,log_id,log_timestamp,user_name,user_id,user_comment,action_type,action,log_title
0,1,2004-12-23T07:51:14Z,Dr.Akarat,174.0,ชาวไอนุ บนเกาะฮอกไกโด ประเทศญี่ปุ่น เมื่อ ค.ศ....,upload,upload,ไฟล์:AinuGroup.jpg
1,2,2004-12-23T09:59:44Z,Dr.Akarat,174.0,เมืองฮาโกดาเตะยามค่ำคืน,upload,upload,ไฟล์:Hakodate.jpg
2,3,2004-12-23T14:54:19Z,Ahoerstemeier,14.0,เนื้อหาเดิม: 'วิวัฒนาการของระบบปฏิบัติการ Linux',delete,delete,รายชื่อระบบปฏิบัติการ


# Exploratory Data Analysis

## Exploring Action Type

In [78]:
df_user_log.action_type.unique()

array(['upload', 'delete', 'block', 'rights', 'move', 'renameuser',
       'protect', 'newusers', 'patrol', 'thanks', 'massmessage', 'merge',
       'create', 'contentmodel', 'gblblock', 'managetags', 'tag',
       'growthexperiments'], dtype=object)

From list of action types, there are 2 action types which could be indicated as a threat to Wikipedia community: <code>'block'</code>, <code>'gblblock'</code>

## Drill down into each Action + comments

In [92]:
df_user_log.loc[df_user_log['action_type'].isin(['block', 'gblblock']), 'action'].unique()

array(['block', 'unblock', 'reblock', 'whitelist', 'gblock2'],
      dtype=object)

When drill down into all actions in these 2 action types, we can see some actions which might not related to bad user behaviors: <code>'unblock'</code>, <code>'whitelist'</code>

In [110]:
df_user_log.loc[df_user_log['action'] == 'unblock', 'user_comment'].unique()[:30]

array(['unblock test', None, 'ป้องกันสร้างบทความแล้ว',
       'รุ่นเก่าหยุดทำงานแล้ว', 'บล็อกผิด', 'ไม่มีปัญหา หรือแก้ไขแล้ว',
       'เปิดโอกาสให้มาเล่น ถึงเวลา 22.00 น.', 'เพื่อปรับระดับการบล็อก',
       '[[แม่แบบ:บล็อกผู้ก่อกวน]]  ใช้ได้ผลดี',
       'ทดสอบแล้ว ไม่เห็นจะต่างจากการบล็อกปกติตรงไหนเลย???',
       'ลดเหลือ 30 นาที', 'ลองอัปเดตโค้ดไปรุ่นใหม่แล้ว',
       'อาจเกิดการบล็อกพลาด', 'เปลี่ยนระยะเวลา เดี๋ยวขาดใจตายเสียก่อน',
       'น่าจะบล็อกผิด', 'อาจเป็นการบล็อกผิดพลาด', 'ไม่ได้ก่อกวน',
       'เหมือนระบบมีปัญหา พ่วงตัวอื่นด้วย', 'อาจเกิดผิดพลาด', 'ผิดพลาด',
       'อาจเกิดความผิดพลาด (ใส่แค่ลิงก์ข้ามภาษา)', 'บล็อกโดยบอตผิดพลาด',
       'อาจเกิดความผิดพลาด', 'ไม่ใช่การก่อกวน', 'IP นี้ใช้หลายที่ในจุฬาฯ',
       'มีเรื่องกับบอตครับ', 'bot ผิดพลาด',
       'ต้องการลบหน้าอย่างผิดวิธี เป็นครั้งแรก', 'บอตผิด', 'บอตพลาด'],
      dtype=object)

For <code>'unblock'</code> type of action, some comments are related to misunderstandings and system error.

In [101]:
df_user_log.loc[df_user_log['action'] == 'whitelist', 'user_comment'].unique()[:20]

array(['Botkung affected', 'prevent collateral damage', 'ใช้โดยบอตคุง'],
      dtype=object)

In [114]:
print("number of records: {}".format(df_user_log.loc[df_user_log['action'] == 'whitelist', :].shape[0]))

number of records: 6


For <code>'whitelist'</code> type of action, available information are not significant (and all comments are related to bot usage, not related to bead behaviors).

In [115]:
df_user_log.loc[df_user_log['action'].isin(('block', 'reblock')), 'user_comment'].unique()[:30]

array(['obscene words', 'spam', 'use obscene word', 'vandal',
       'repeat nonsense edit', 'เขียนเนื้อหาผิดนโยบายของวิกิพีเดีย', None,
       'ผิดนโยบาย', 'testing bug?', 'testing bug', 'block test',
       'ก่อกวนบ่อยครั้ง', 'ป่วน และ อาจก่อให้เกิดการขัดแย้ง',
       'repeat spam', 'vandalism',
       'ก่อกวน 3 บทความ [[ฮิวงะ ฮินาตะ]], ฯลฯ', 'ก่อกวน',
       'ก่อกวนซ้ำหลังจากแจ้งเตือนแล้ว', 'Impersonating', 'กันการแอบอ้าง',
       'อัปโหลดภาพที่ไม่เกี่ยว (หลังจากมีการเตือน)', 'spammer',
       'โฆษณาเว็บไม่เหมาะสม', 'ป่วน',
       'ก่อกวน *** ON WHEELS!!!!!!!!!!!!!!',
       'Willy on Wheels - needs indefinite block', 'โฆษณา', 'spam sites',
       '3RR violation', 'spam รอบ 3'], dtype=object)

For <code>'block'</code> & <code>'reblock'</code> type of action, obviously, all sample comments are related to all the troubles caused by users, such as disturbance, spam, offensive words, posting promotions, etc.

In [109]:
df_user_log.loc[df_user_log['action'].isin(('gblock2', )), :]

,log_id,log_timestamp,user_name,user_id,user_comment,action_type,action,log_title
1277713,1379999,2022-02-19T10:03:45Z,Maintenance script,230668.0,[[m:NOP|Open proxy]]: See the [[m:WM:OP/H|help...,gblblock,gblock2,ผู้ใช้:178.48.120.20
1277716,1380002,2022-02-19T10:05:03Z,Maintenance script,230668.0,[[m:NOP|Open proxy]]: See the [[m:WM:OP/H|help...,gblblock,gblock2,ผู้ใช้:49.36.230.64
1277726,1380012,2022-02-19T10:08:41Z,Maintenance script,230668.0,[[m:NOP|Open proxy]]: See the [[m:WM:OP/H|help...,gblblock,gblock2,ผู้ใช้:178.168.42.198


Finally, <code>'gblock2'</code> type of action are not related.

## Exploring behavior of blocked users

In [121]:
df_user_log['log_timestamp'] = df_user_log['log_timestamp'].str.replace('T', ' ')
df_user_log['log_timestamp'] = df_user_log['log_timestamp'].str.replace('Z', '')

df_user_log['log_timestamp'] = pd.to_datetime(df_user_log['log_timestamp'])

In [159]:
list_blocked_user_id = df_user_log.loc[df_user_log['action_type'].isin(('block', 'reblock')), 'user_id'].unique().tolist()

df_user_blocked = df_user_log.loc[df_user_log['user_id'].isin(list_blocked_user_id), :].copy()

In [143]:
print("number of users created within log period: ", df_user_blocked.loc[df_user_blocked.action.eq('newusers'), 'user_id'].nunique())
print("number of all users existed in log :", df_user_blocked.loc[:, 'user_id'].nunique())

number of users created within log period:  5
number of all users existed in log : 48


In [160]:
df_user_blocked['block_timestamp'] = pd.to_datetime(np.where(df_user_blocked['action_type'].isin(('block', 'reblock')), df_user_blocked['log_timestamp'].copy(), pd.NaT))
df_user_blocked.sort_values(by=['user_id', 'log_timestamp'], inplace=True)
df_user_blocked['block_timestamp'] = df_user_blocked.groupby('user_id')['block_timestamp'].bfill()

In [161]:
df_user_blocked

,log_id,log_timestamp,user_name,user_id,user_comment,action_type,action,log_title,block_timestamp
2543,2544,2005-08-29 21:55:27,Brooke Vibber,1.0,testing bug?,block,block,ผู้ใช้:TestBlock,2005-08-29 21:55:27
2544,2545,2005-08-29 22:22:37,Brooke Vibber,1.0,testing bug,block,block,ผู้ใช้:TestBlock,2005-08-29 22:22:37
157222,161699,2008-12-28 18:02:19,Brooke Vibber,1.0,Cut file size from 30k to 16k (converted to gr...,upload,overwrite,ไฟล์:Wiki.png,NaT
2,3,2004-12-23 14:54:19,Ahoerstemeier,14.0,เนื้อหาเดิม: 'วิวัฒนาการของระบบปฏิบัติการ Linux',delete,delete,รายชื่อระบบปฏิบัติการ,2006-05-10 09:42:19
6,7,2004-12-24 10:26:04,Ahoerstemeier,14.0,เนื้อหาเดิม: 'sign laguage',delete,delete,ภาษามือไต้หวัน,2006-05-10 09:42:19
...,...,...,...,...,...,...,...,...,...
1603951,1722747,2024-09-21 14:54:25,Kaoavi,401376.0,เปลี่ยนชื่อ,move,move_redir,แรม,NaT
1603952,1722748,2024-09-21 14:54:25,Kaoavi,401376.0,เปลี่ยนชื่อ,move,move,พูดคุย:แรม,NaT
1603953,1722749,2024-09-21 15:05:03,Kaoavi,401376.0,"เนื้อหาเดิม: ""{{ลบ|ไม่จำเป็น}}""",delete,delete,คุยเรื่องวิกิพีเดีย:EF,NaT
1603954,1722750,2024-09-21 15:08:44,Kaoavi,401376.0,ไม่จำเป็นแล้ว,protect,unprotect,ค้นหา,NaT


# Proposed Solution

<u><b>Solution</b></u> : ML solution that enable <b>proactive user blocking</b> by predicting the change of each particular user getting blocked by other users (in the next n days) based on the previous usage behavior, blocking history, and so on.

<u><b>Data</b></u> : log usage level (aggregated previous usage by determined time frame)

<u><b>Who use this</b></u> : Wikipedia web admin / admin users / system

<u><b>How to use it</b></u> : Use the solution to predict the probability 